# 執行腳本

In [ ]:
%load_ext autoreload
%autoreload 2

import data_loader
import data_preprocessing
import feature_eng
# import model_train

# ------------------------------------------------
# Step 1: 載入 (Load) - 只管讀進來
# ------------------------------------------------
df_raw = data_loader.load_and_merge_data()

# ------------------------------------------------
# Step 2: 前處理 (Preprocess) - 只管洗乾淨
# ------------------------------------------------
df_clean = data_preprocessing.clean_data(df_raw)

# ------------------------------------------------
# Step 3: 特徵工程 (Feature) - 只管算指標
# ------------------------------------------------
# 注意：用洗乾淨的資料來算指標，才不會報錯
df_features = feature_eng.add_technical_indicators(df_clean)

# ------------------------------------------------
# Step 4: 訓練 (Train) - 切分、正規化、訓練
# ------------------------------------------------
# 正規化 (Normalization) 還是建議放在這裡，跟著切分一起做比較安全
# X_train, y_train, X_test, y_test, scaler = model_train.preprocess_data(df_features)

print("🎉 整個流程執行完畢！")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🚀 [Loader] 開始讀取資料...
✅ 資料載入完成 (尚未清洗)！大小：(852, 12)
🧹 [Preprocessing] 開始清洗資料...
✅ 資料清洗完成！沒有 NaN 了。
📊 開始特徵工程 (MA, RSI, MACD)，共處理 11 支股票...
✅ 特徵工程完成！總欄位數: 89
🎉 整個流程執行完畢！
